## PARCEIROS 360 - COMISSAO

###  Pagamento - Utilização Factoring & Bloqueios

- <b>Analista(s)</b>: Daniel Rezende
- <b>Projeto</b>: Parceiros 360 - Um modelo de análise de risco para auxiliar no controle das tomadas de decisões através de análises preditivas
- <b>Data</b>: 09/12/2022
- <b>Descrição</b>: 

In [58]:
!python --version

Python 3.8.16


### BIBLIOTECAS:

In [59]:
# !pip install treeinterpreter

In [60]:
## data
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from unicodedata import normalize

## vis
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## modelo
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

from treeinterpreter import treeinterpreter as ti

## alertas
import warnings
warnings.filterwarnings("ignore")

### LAYOUT:

In [61]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

rc={'font.size': 16, 'axes.labelsize': 16, 'legend.fontsize': 16,
    'axes.titlesize': 18, 'xtick.labelsize': 16, 'ytick.labelsize': 16}

sns.set(rc=rc)
sns.set_style("darkgrid")
sns.set_palette("pastel")

### CONFIGS:

In [62]:
#pd.set_option('max_columns', 200)
pd.options.display.max_columns = 200 

seed = 42
np.random.seed(seed)

### FUNÇÕES

In [63]:
#SE PRECISAR DO ONE HOT USAR ANTES DO SCALERS PRA NAO DAR ERRO

def scalers(base):
    nova_base = base.copy()
    scaler = MinMaxScaler(feature_range=(0,1))
    for column in base.columns:
        aux = scaler.fit_transform(base[[column]])
        nova_base[column] = aux
    return nova_base

In [64]:
#base_treino_scaled = BASE DE TREINO, SEM A COLUNA DO KPI
#DADOS = BASE COMPLETA, COM O TARGET
#KPI NOME = O NOME DA COLUNA ONDE ESTÁ O TARGET

def makedf(base_treino_scaled, dados, kpi_nome, base_treino):
    
    kmeans = KMeans(n_clusters = 3, max_iter = 300, init = 'k-means++', n_init = 10, random_state = 0)
    clusters = kmeans.fit_predict(base_treino_scaled)

    model = RandomForestClassifier(random_state=0)
    model.fit(base_treino_scaled,clusters)
    prediction, bias, contributions = ti.predict(model, base_treino_scaled)

    contributions = np.round(contributions,5)*1000

    contributions_c1 = []
    contributions_c2 = []
    contributions_c3 = []
    contributions_filtered = []

    for i in range(len(contributions)):
        contributions_c1.append([contributions[i][0][0], contributions[i][1][0]])
        contributions_c2.append([contributions[i][0][1], contributions[i][1][1]])
        contributions_c3.append([contributions[i][0][2], contributions[i][1][2]])
    
    for i in range(len(clusters)):
        if clusters[i] == 0:
            contributions_filtered.append(contributions_c1[i])
        if clusters[i] == 1:
            contributions_filtered.append(contributions_c2[i])
        if clusters[i] == 2:
            contributions_filtered.append(contributions_c3[i])

    scaler = MinMaxScaler(feature_range=(0,100))
    contributions_filtered_scaled = scaler.fit_transform(contributions_filtered)

    contributions_filtered = pd.DataFrame(contributions_filtered_scaled, columns=base_treino_scaled.columns + '_CONTRIBUTION')

    base_treino['KPI_original'] = dados[kpi_nome].values
    base_treino['resultado_modelo'] = clusters
    base_treino['resultado_modelo'] = base_treino['resultado_modelo'].map({0:'RISCO BAIXO', 1:'RISCO MÉDIO', 2:'RISCO ALTO'})

    base_treino.reset_index(inplace=True, drop=False)

    results = pd.concat([base_treino, contributions_filtered], axis=1)
    return results

### DADOS:

In [65]:
PATH = "intermediate/"

***
### COMISSÃO: 

In [66]:
FILE = "etl_previsao_comissao.xlsx"
etl_previsao_comissao = pd.read_excel(PATH + FILE)
etl_previsao_comissao['RAIZ_CNPJ'] = etl_previsao_comissao['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
etl_previsao_comissao = etl_previsao_comissao.drop(['Unnamed: 0'], axis=1)
# etl_previsao_comissao = etl_previsao_comissao.drop(['Unnamed: 0', 'FORNECEDOR', 'AREA_ATUACAO'], axis=1)

etl_previsao_comissao = etl_previsao_comissao.drop_duplicates().reset_index(drop=True)

etl_previsao_comissao

,RAIZ_CNPJ,COMPETENCIA,INDICADOR_RISCO_FIN,FATOR_RISCO,KPI_comissao,NOME_FORNECEDOR,INDICADOR_RISCO_FIN_OE,FATOR_RISCO_OE
0,78599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9
1,78599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO MÉDIO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9
2,78599,2021-02-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9
3,78599,2021-02-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO MÉDIO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9
4,78599,2021-03-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9
...,...,...,...,...,...,...,...,...
16274,46298215,2022-07-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,SOUZA E SOUZA INTER. EM TELEFONIA L,3,9
16275,46337103,2022-06-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,R E COM E SER DE TELEFONIA LTDA,3,9
16276,46337103,2022-07-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,R E COM E SER DE TELEFONIA LTDA,3,9
16277,46354697,2022-06-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,ALLIANCE TELECOMUNICACOES BRASIL LT,3,9


In [67]:
dados = etl_previsao_comissao.copy()
dados.set_index(['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], inplace=True)
dados

INDICADOR_RISCO_FIN      FATOR_RISCO  KPI_comissao  INDICADOR_RISCO_FIN_OE  FATOR_RISCO_OE
RAIZ_CNPJ COMPETENCIA NOME_FORNECEDOR                                                                                                                  
78599     2021-01-01  GERENCIAL BRASIL PARTICIPAÃÃES E EM  SEM INDICADOR RISCO  SEM FATOR RISCO   RISCO BAIXO                       3               9
                      GERENCIAL BRASIL PARTICIPAÃÃES E EM  SEM INDICADOR RISCO  SEM FATOR RISCO   RISCO MÉDIO                       3               9
          2021-02-01  GERENCIAL BRASIL PARTICIPAÃÃES E EM  SEM INDICADOR RISCO  SEM FATOR RISCO   RISCO BAIXO                       3               9
                      GERENCIAL BRASIL PARTICIPAÃÃES E EM  SEM INDICADOR RISCO  SEM FATOR RISCO   RISCO MÉDIO                       3               9
          2021-03-01  GERENCIAL BRASIL PARTICIPAÃÃES E EM  SEM INDICADOR RISCO  SEM FATOR RISCO   RISCO BAIXO                       3               9
...                                                                          ...              ...           ...                     ...             ...
46298215  2022-07-01  SOUZA E SOUZA INTER. EM TELEFONIA L    SEM INDICADOR RISCO  SEM FATOR RISCO  NÃO AVALIADO                       3               9
46337103  2022-06-01  R E COM E SER DE TELEFONIA LTDA        SEM INDICADOR RISCO  SEM FATOR RISCO  NÃO AVALIADO                       3               9
          2022-07-01  R E COM E SER DE TELEFONIA LTDA        SEM INDICADOR RISCO  SEM FATOR RISCO  NÃO AVALIADO                       3               9
46354697  2022-06-01  ALLIANCE TELECOMUNICACOES BRASIL LT    SEM INDICADOR RISCO  SEM FATOR RISCO  NÃO AVALIADO                       3               9
          2022-07-01  ALLIANCE TELECOMUNICACOES BRASIL LT    SEM INDICADOR RISCO  SEM FATOR RISCO  NÃO AVALIADO                       3               9

[16279 rows x 5 columns]

In [68]:
dados.columns

Index(['INDICADOR_RISCO_FIN', 'FATOR_RISCO', 'KPI_comissao', 'INDICADOR_RISCO_FIN_OE', 'FATOR_RISCO_OE'], dtype='object')

In [69]:
base_treino_kpi = dados[['INDICADOR_RISCO_FIN_OE', 'FATOR_RISCO_OE']]

In [70]:
dados

INDICADOR_RISCO_FIN      FATOR_RISCO  KPI_comissao  INDICADOR_RISCO_FIN_OE  FATOR_RISCO_OE
RAIZ_CNPJ COMPETENCIA NOME_FORNECEDOR                                                                                                                  
78599     2021-01-01  GERENCIAL BRASIL PARTICIPAÃÃES E EM  SEM INDICADOR RISCO  SEM FATOR RISCO   RISCO BAIXO                       3               9
                      GERENCIAL BRASIL PARTICIPAÃÃES E EM  SEM INDICADOR RISCO  SEM FATOR RISCO   RISCO MÉDIO                       3               9
          2021-02-01  GERENCIAL BRASIL PARTICIPAÃÃES E EM  SEM INDICADOR RISCO  SEM FATOR RISCO   RISCO BAIXO                       3               9
                      GERENCIAL BRASIL PARTICIPAÃÃES E EM  SEM INDICADOR RISCO  SEM FATOR RISCO   RISCO MÉDIO                       3               9
          2021-03-01  GERENCIAL BRASIL PARTICIPAÃÃES E EM  SEM INDICADOR RISCO  SEM FATOR RISCO   RISCO BAIXO                       3               9
...                                                                          ...              ...           ...                     ...             ...
46298215  2022-07-01  SOUZA E SOUZA INTER. EM TELEFONIA L    SEM INDICADOR RISCO  SEM FATOR RISCO  NÃO AVALIADO                       3               9
46337103  2022-06-01  R E COM E SER DE TELEFONIA LTDA        SEM INDICADOR RISCO  SEM FATOR RISCO  NÃO AVALIADO                       3               9
          2022-07-01  R E COM E SER DE TELEFONIA LTDA        SEM INDICADOR RISCO  SEM FATOR RISCO  NÃO AVALIADO                       3               9
46354697  2022-06-01  ALLIANCE TELECOMUNICACOES BRASIL LT    SEM INDICADOR RISCO  SEM FATOR RISCO  NÃO AVALIADO                       3               9
          2022-07-01  ALLIANCE TELECOMUNICACOES BRASIL LT    SEM INDICADOR RISCO  SEM FATOR RISCO  NÃO AVALIADO                       3               9

[16279 rows x 5 columns]

In [71]:
base_treino_kpi

INDICADOR_RISCO_FIN_OE  FATOR_RISCO_OE
RAIZ_CNPJ COMPETENCIA NOME_FORNECEDOR                                                              
78599     2021-01-01  GERENCIAL BRASIL PARTICIPAÃÃES E EM                       3               9
                      GERENCIAL BRASIL PARTICIPAÃÃES E EM                       3               9
          2021-02-01  GERENCIAL BRASIL PARTICIPAÃÃES E EM                       3               9
                      GERENCIAL BRASIL PARTICIPAÃÃES E EM                       3               9
          2021-03-01  GERENCIAL BRASIL PARTICIPAÃÃES E EM                       3               9
...                                                                             ...             ...
46298215  2022-07-01  SOUZA E SOUZA INTER. EM TELEFONIA L                         3               9
46337103  2022-06-01  R E COM E SER DE TELEFONIA LTDA                             3               9
          2022-07-01  R E COM E SER DE TELEFONIA LTDA                             3               9
46354697  2022-06-01  ALLIANCE TELECOMUNICACOES BRASIL LT                         3               9
          2022-07-01  ALLIANCE TELECOMUNICACOES BRASIL LT                         3               9

[16279 rows x 2 columns]

In [72]:
base_treino_kpi_scaled = scalers(base_treino_kpi)

base_treino_kpi_scaled

INDICADOR_RISCO_FIN_OE  FATOR_RISCO_OE
RAIZ_CNPJ COMPETENCIA NOME_FORNECEDOR                                                              
78599     2021-01-01  GERENCIAL BRASIL PARTICIPAÃÃES E EM                     1.0             1.0
                      GERENCIAL BRASIL PARTICIPAÃÃES E EM                     1.0             1.0
          2021-02-01  GERENCIAL BRASIL PARTICIPAÃÃES E EM                     1.0             1.0
                      GERENCIAL BRASIL PARTICIPAÃÃES E EM                     1.0             1.0
          2021-03-01  GERENCIAL BRASIL PARTICIPAÃÃES E EM                     1.0             1.0
...                                                                             ...             ...
46298215  2022-07-01  SOUZA E SOUZA INTER. EM TELEFONIA L                       1.0             1.0
46337103  2022-06-01  R E COM E SER DE TELEFONIA LTDA                           1.0             1.0
          2022-07-01  R E COM E SER DE TELEFONIA LTDA                           1.0             1.0
46354697  2022-06-01  ALLIANCE TELECOMUNICACOES BRASIL LT                       1.0             1.0
          2022-07-01  ALLIANCE TELECOMUNICACOES BRASIL LT                       1.0             1.0

[16279 rows x 2 columns]

### CLUSTERING COM K-Means:

In [73]:
#base_treino_scaled = BASE DE TREINO, SEM A COLUNA DO KPI, scaled
#DADOS = BASE COMPLETA, COM O TARGET
#KPI NOME = O NOME DA COLUNA ONDE ESTÁ O TARGET
#base_treino_kpi = BASE DE TREINO, SEM A COLUNA DO KPI


results_kpi = makedf(base_treino_kpi_scaled, dados, 'KPI_comissao', base_treino_kpi)
results_kpi

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,INDICADOR_RISCO_FIN_OE,FATOR_RISCO_OE,KPI_original,resultado_modelo,INDICADOR_RISCO_FIN_OE_CONTRIBUTION,FATOR_RISCO_OE_CONTRIBUTION
0,78599,2021-01-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9,RISCO BAIXO,RISCO MÉDIO,100.0,8.306264
1,78599,2021-01-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9,RISCO MÉDIO,RISCO MÉDIO,100.0,8.306264
2,78599,2021-02-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9,RISCO BAIXO,RISCO MÉDIO,100.0,8.306264
3,78599,2021-02-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9,RISCO MÉDIO,RISCO MÉDIO,100.0,8.306264
4,78599,2021-03-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9,RISCO BAIXO,RISCO MÉDIO,100.0,8.306264
...,...,...,...,...,...,...,...,...,...
16274,46298215,2022-07-01,SOUZA E SOUZA INTER. EM TELEFONIA L,3,9,NÃO AVALIADO,RISCO MÉDIO,100.0,8.306264
16275,46337103,2022-06-01,R E COM E SER DE TELEFONIA LTDA,3,9,NÃO AVALIADO,RISCO MÉDIO,100.0,8.306264
16276,46337103,2022-07-01,R E COM E SER DE TELEFONIA LTDA,3,9,NÃO AVALIADO,RISCO MÉDIO,100.0,8.306264
16277,46354697,2022-06-01,ALLIANCE TELECOMUNICACOES BRASIL LT,3,9,NÃO AVALIADO,RISCO MÉDIO,100.0,8.306264


In [74]:
results_kpi = results_kpi.drop_duplicates().reset_index(drop=True)
results_kpi
# results_kpi = results_kpi.tail(results_kpi.shape[0] -215177) 16000
# results_kpi

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,INDICADOR_RISCO_FIN_OE,FATOR_RISCO_OE,KPI_original,resultado_modelo,INDICADOR_RISCO_FIN_OE_CONTRIBUTION,FATOR_RISCO_OE_CONTRIBUTION
0,78599,2021-01-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9,RISCO BAIXO,RISCO MÉDIO,100.0,8.306264
1,78599,2021-01-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9,RISCO MÉDIO,RISCO MÉDIO,100.0,8.306264
2,78599,2021-02-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9,RISCO BAIXO,RISCO MÉDIO,100.0,8.306264
3,78599,2021-02-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9,RISCO MÉDIO,RISCO MÉDIO,100.0,8.306264
4,78599,2021-03-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,3,9,RISCO BAIXO,RISCO MÉDIO,100.0,8.306264
...,...,...,...,...,...,...,...,...,...
16274,46298215,2022-07-01,SOUZA E SOUZA INTER. EM TELEFONIA L,3,9,NÃO AVALIADO,RISCO MÉDIO,100.0,8.306264
16275,46337103,2022-06-01,R E COM E SER DE TELEFONIA LTDA,3,9,NÃO AVALIADO,RISCO MÉDIO,100.0,8.306264
16276,46337103,2022-07-01,R E COM E SER DE TELEFONIA LTDA,3,9,NÃO AVALIADO,RISCO MÉDIO,100.0,8.306264
16277,46354697,2022-06-01,ALLIANCE TELECOMUNICACOES BRASIL LT,3,9,NÃO AVALIADO,RISCO MÉDIO,100.0,8.306264


In [75]:
results_kpi.to_excel(r'outputs/previsao_comissao.xlsx')